In [14]:
from src.classes.network import RandomNetwork, ScaleFreeNetwork
from src.classes.node import Node
from src.experimentation import generate_networks, read_and_load_networks, multiple_correlations_par
from src.viusalization import plot_cascade_animation, test_significance, calculate_average_per_gamma, plot_cascades_gamma #,plot_cascade_dist_average
from collections import defaultdict
import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
from collections import defaultdict
import os, sys
import numpy as np



#### Global Values

In [2]:
correlations = np.linspace(-1, 1, 11)
correlations = np.round(correlations, 1)
initial_seeds = np.linspace(13, 1600, 11)
num_runs = 30
num_nodes = 200
update_fraction = 0.1
average_degree = 8
starting_distribution = 0.5     # L / R ratio (niet per se nodig maar kan misschien leuk zijn om te varieern)
p = average_degree/(num_nodes-1) 
updates = 300000
m = 4

# vary between random and scale_free
# what_net = "random"
what_net = "scale_free"

### Hoi dit is ff uitleg voor jullie voor de nieuwe classes:

Toevoegen van ScaleFreeNetwork

In [3]:
# # Gebruik plot=True om begin plot van distributie te zien
# network = ScaleFreeNetwork(m=m, plot=True)

# for round in range(10000):
#     network.update_round()

# # Gebruik deze functie om de distributie te plotten op het einde
# network.verify_scale_free_distribution(plot=True)

Toevoegen van RandomNetwork

In [4]:
# IPV Network() aan te roepen doe je nu RandomNetwork() of ScaleFreeNetwork()
# # Je kunt rustig dezelfde argumenten meegeven als bij Network() zoals p=0.1, k=8. Als dit wordt leeggelaaten worden standaard waarden gebruikt.

# random_network = RandomNetwork()
# for round in range(10000):
#     random_network.update_round()

#### Developing and saving network

In [ ]:
# # dummy values
# num_runs = 10
# updates=1000
# # scale-free
# generate_networks(correlations, initial_seeds, num_nodes=num_nodes, iterations=updates, how_many=num_runs, update_fraction=update_fraction, starting_distribution=starting_distribution, p=p, network_sort=what_net, m=m)

 #### Reading in and generating Network

In [6]:
# # # dummy values
# # num_runs = 5
# # updates=1000
# test=False


# # Read in the network and save it in a datastructure
# # all_networks = read_and_load_networks(num_runs, num_nodes, update_fraction, average_degree, starting_distribution, correlations, whichtype=what_net)
# all_networks = read_and_load_networks(num_runs, num_nodes, update_fraction, average_degree, starting_distribution, correlations, whichtype="random")

# # test for consistency of the saved network
# if test:
#     used_seed = int(initial_seeds[0])
#     if what_net == "scale_free":
#         test_network = ScaleFreeNetwork(num_nodes=num_nodes, m=m, mean=0, correlation=-1.0, update_fraction=update_fraction, starting_distribution=starting_distribution, seed=used_seed)    ### !!!!!!!!!!!!!!
#     else: 
#         test_network = RandomNetwork(num_nodes=num_nodes, mean=0, correlation=-1.0, update_fraction=update_fraction, starting_distribution=starting_distribution, seed=used_seed, p=p)
#     number_of_alterations = 0


#     assert set([(conn[0].ID, conn[1].ID) for conn in all_networks[(-1.0, 0)][0].connections]) == set([(conn[0].ID, conn[1].ID) for conn in test_network.connections]), "The networks that are generated should be the same at the start"

#     for _ in range(updates):
#         test_network.update_round()
#         number_of_alterations += test_network.alterations
#         test_network.clean_network()    
        
#     assert set([(conn[0].ID, conn[1].ID) for conn in all_networks[(-1.0, 0)][1].connections]) == set([(conn[0].ID, conn[1].ID) for conn in test_network.connections]), "The networks that are generated should be the same at the end"



#### Analyzing and plotting the Cascade distributions

## Calculating the cascades (parallelized implementation)
### Reading in network and saving everything in datastructures. structured by size and correlation value. 


In [ ]:

# datastructures
cascades_before = defaultdict(lambda: defaultdict(list))
cascades_after = defaultdict(lambda: defaultdict(list))
cascades_before_averaged = defaultdict(lambda: defaultdict(list))
cascades_after_averaged = defaultdict(lambda: defaultdict(list))
save=True
sizes = defaultdict()
sizes_averaged = defaultdict()

cascades_before_averaged_rand = defaultdict(lambda: defaultdict(list))
cascades_after_averaged_rand = defaultdict(lambda: defaultdict(list))
sizes_averaged_rand = defaultdict()




# run the cascades for different correlations (for both the initial and updated network), saving the cascade polarizations and cascade sizes in a dictionary
for corr in correlations: 
    print(f"starting experimentation for correlation: {corr}")
    print("-----------------------------------------------")

    (before_after, before_after_averaged, largest_sizes) = multiple_correlations_par(corr, num_runs, num_nodes, update_fraction, average_degree, starting_distribution,what_net)
    (collection_of_all_before, collection_of_all_after) = before_after
    (coll_of_all_before_averaged, coll_of_all_after_averaged) = before_after_averaged
    (largest_size_of_all, largest_size_of_all_averaged) = largest_sizes
    
    # sizes[corr] = largest_size_of_all
    # cascades_before[corr] = collection_of_all_before
    # cascades_after[corr] = collection_of_all_after

    sizes_averaged[corr] = largest_size_of_all_averaged
    cascades_before_averaged[corr] = coll_of_all_before_averaged
    cascades_after_averaged[corr] = coll_of_all_after_averaged


for corr in correlations: 
    print(f"starting experimentation for correlation: {corr} (random)")
    print("-----------------------------------------------")

    (before_after, before_after_averaged, largest_sizes) = multiple_correlations_par(corr, num_runs, num_nodes, update_fraction, average_degree, starting_distribution,"random")
    (collection_of_all_before, collection_of_all_after) = before_after
    (coll_of_all_before_averaged, coll_of_all_after_averaged) = before_after_averaged
    (largest_size_of_all, largest_size_of_all_averaged) = largest_sizes
    
    sizes_averaged_rand[corr] = largest_size_of_all_averaged
    cascades_before_averaged_rand[corr] = coll_of_all_before_averaged
    cascades_after_averaged_rand[corr] = coll_of_all_after_averaged



#### Annimation of cascade size distribution with average polarization. (average cascade size per sampled node is calculated)

In [ ]:
plot_cascade_animation(cascades_before_averaged, cascades_after_averaged, list(reversed(correlations)), sizes_averaged, num_runs, what_net, save=True, averaged=True)
plot_cascade_animation(cascades_before_averaged_rand, cascades_after_averaged_rand, list(reversed(correlations)), sizes_averaged_rand, num_runs, "random", save=True, averaged=True)


In [42]:
def plot_cascades_gamma(cas, num_runs, what_net):
    values_bef, values_af, variance_bef, variance_af = calculate_average_per_gamma(cas, num_runs)
    fig, ax = plt.subplots(figsize=(5,4))

    green_to_red = plt.cm.RdYlGn_r
    # Normalize polarization for coloring
    keys_bef = list(values_bef.keys())  # Gamma values (X-axis)
    sizes_bef = [v[0] for v in values_bef.values()]  # Sizes for Y-axis
    pol_bef = [v[1] for v in values_bef.values()]  # Polarization for colors

    keys_af = list(values_af.keys())  # Gamma values (X-axis)
    sizes_af = [v[0] for v in values_af.values()]  # Sizes for Y-axis
    pol_af = [v[1] for v in values_af.values()]  # Polarization for colors

    #Coloring scheme with polarization
    norm = plt.Normalize(vmin=0, vmax=1)
    colors_bef = [green_to_red(norm(p)) for p in pol_bef]
    colors_af = [green_to_red(norm(p)) for p in pol_af]
    
    var_sizes_bef = np.array([variance_bef[k][0] for k in keys_bef])  # Variance for sizes (Before)
    var_sizes_af = np.array([variance_af[k][0] for k in keys_af])  # Variance for sizes (After)

    #Computing confidence interval with calculated variance
    sem_sizes_bef = np.sqrt(var_sizes_bef) / np.sqrt(num_runs)
    sem_sizes_af = np.sqrt(var_sizes_af) / np.sqrt(num_runs)
    ci_sizes_bef = 1.96 * sem_sizes_bef
    ci_sizes_af = 1.96 * sem_sizes_af

    sm = plt.cm.ScalarMappable(cmap=green_to_red, norm=plt.Normalize(vmin=0, vmax=1))
    sm.set_array([])
    cbar = plt.colorbar(sm, ax=ax)
    cbar.set_label("0:non polarized-1:fully polarized")
    
    
    # make an errorbar to visualize CI
    ax.errorbar(keys_bef, sizes_bef, yerr=ci_sizes_bef, color="black", linewidth = 0.8, capsize=2, label="", linestyle="none", zorder=1)
    ax.errorbar(keys_af, sizes_af, yerr=ci_sizes_af, color="black", linewidth=0.8, capsize=2, label="", linestyle="none", zorder=1)

    if what_net != "both":
        label1 = "Before"
        label2 = "After"
        marker1 = "o"
        marker2 = "s"
    else:
        label1 = "Random"
        label2 = "Scale-free"
        marker1 = "v"
        marker2 = "h"
    for x, y, color in zip(keys_bef, sizes_bef, colors_bef):
        ax.scatter(x, y, color=color, edgecolor="black", linewidths=0.5, marker=marker1, s=50, label="Before" if x == keys_bef[0] else "", zorder=2)

    for x, y, color in zip(keys_af, sizes_af, colors_af):
        ax.scatter(x, y, color=color, edgecolor="black", linewidths=0.5, marker=marker2, s=50, label="After" if x == keys_af[0] else "", zorder=2)
    
    
    legend_handles = [
    plt.Line2D([], [], marker=marker1, color="w", markerfacecolor="white", markersize=6, markeredgecolor="black", label=label1),
    plt.Line2D([], [], marker=marker2, color="w", markerfacecolor="white", markersize=6, markeredgecolor="black", label=label2),
    ]
    ax.set_xlabel("Correlation")
    ax.set_ylabel("Average Cascade Size")
    # ax.set_title("Cascade Size Before and After vs. Gamma")
    ax.legend(handles=legend_handles)

    plt.savefig(f"plots/cascade_distribution/{what_net}/averaged_over_gammas.png", dpi=300, bbox_inches='tight')
    plt.show()

##### Summarizing all cascade info in one plot 

In [ ]:
# for key,value in cascades_before_averaged.items():
#     plot_cascade_dist_average(value, "before", largest_size_of_all_averaged, num_runs, save, key)

# for key,value in cascades_after_averaged.items():
#     plot_cascade_dist_average(value, "after", largest_size_of_all_averaged, num_runs, save, key)


plot_cascades_gamma((cascades_before_averaged, cascades_after_averaged), num_runs, what_net)
plot_cascades_gamma((cascades_after_averaged_rand, cascades_after_averaged), num_runs, "both")
plot_cascades_gamma((cascades_before_averaged_rand, cascades_after_averaged_rand), num_runs, "random")

#### Calculating significance between cascade sizes and polarizations between networks before and after updating, for different gamma values

In [12]:
which_cas = [(cascades_before_averaged, cascades_after_averaged), (cascades_before_averaged_rand, cascades_after_averaged_rand), (cascades_after_averaged_rand, cascades_after_averaged)]

for i, what in enumerate(["scale_free", "random", "both"]):
    values_bef, values_af, variance_bef, variance_af = calculate_average_per_gamma(which_cas[i], num_runs)
    results = test_significance(values_bef, values_af, variance_bef, variance_af, num_runs)
    output_file = f"statistics/cascades/results_bef_af_{what}.txt"

    with open(output_file, "w") as f:
        f.write(f"Statistical significance for {what} network type (cascade experiments)\n")
        for gamma, res in results.items():
            f.write(f"Gamma = {gamma}:\n")
            f.write(f"  Size: t = {res['t_size']:.3f}, p = {res['p_size']:.3g}\n")
            f.write(f"  Polarization: t = {res['t_pol']:.3f}, p = {res['p_pol']:.3g}\n")
            f.write("--------------------------------------------------------\n")
